# Launch a Jupyter Lab notebook in appmode

[Appmode](https://github.com/oschuett/appmode) is useful for displaying Jupyter notebooks as web apps, with all the code hidden. But you can't use `appmode` within Jupyter Lab. Here's some code to create a button that launches the current notebook in `appmode`. It works in either Jupyter Lab or the classic notebook view. If `appmode` isn't available it does nothing.

In [ ]:
from notebook.notebookapp import list_running_servers
import requests
from requests.compat import urljoin
import json
import os
import ipykernel
import re
from IPython.display import display, HTML

def create_appmode_link():
    '''
    This creates a link to run the current notebook in appmode.
    This is mainly useful in Jupyter Lab where appmode won't work.
    Clicking on the link in Lab will take you into the classic notebook environment and run appmode.
    Modified from: https://github.com/jupyter/notebook/issues/1000
    '''
    try:
        import appmode
    except ImportError:
        pass
    else:
        kernel_id = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
        servers = list_running_servers()
        for ss in servers:
            response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
            for nn in json.loads(response.text):
                if nn['kernel']['id'] == kernel_id:
                    relative_path = nn['notebook']['path']
                    # Notebooks running in appmode are prefixed with '.' so if it's already in appmode we'll do nothing
                    if relative_path[0] != '.':
                        url = os.path.join(ss['base_url'], 'apps', relative_path)
                        display(HTML('<p>This notebook was designed to run in Appmode (with all the code hidden).</p>'))
                        display(HTML('<a style="padding: 5px; display:block; width: 200px; text-align: center; background-color: #2196F3; color: white;" href="{}">Launch in appmode</a>'.format(url)))
                    else:
                        display(HTML('<p>This is a Jupyter notebook running in <a href="https://glam-workbench.github.io/getting-started/#running-notebooks-as-apps">App mode</a> (with all the code hidden). To view and edit the code, click on the <b>Edit App</b> button.</p>'))

create_appmode_link()